In [62]:
import pandas as pd 
import json
from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#from plotly.offline import iplot

# enable plotly in VS Studio Code
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook" 

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths
from workbench.wandb import wandb_model_DB, get_model_DB_run_id_from_architecture, get_architecture_from_model_DB_run_id, get_wandb_table_as_df

import wandb
%load_ext autoreload



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
%reload_ext autoreload
%autoreload

In [3]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [4]:
models_dir = initialize()

In [66]:
automated = False

global model_name

#model_name = "mobilenetv3smallNSQ_0.25_96_c3_o2_se4l128.MV3"
model_name = "mobilenetv1vvw_0.25_96_c3_o2_vvw"



models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

c:\tiny_mlc\tiny_cnn\models


In [67]:
models_stm32_benchmark_file = models_dir.joinpath(model_name, "network_output", "network_validate_report.txt")
models_stm32_benchmark_file
models_stm32_benchmark_file_INT8 = models_dir.joinpath(model_name, "network_output", "network_validate_report_INT8.txt")
models_stm32_benchmark_file_INT8

WindowsPath('c:/tiny_mlc/tiny_cnn/models/mobilenetv1vvw_0.25_96_c3_o2_vvw/network_output/network_validate_report_INT8.txt')

# Helper functions

In [68]:
def string_percent_to_float(x):
    x = str(x).strip("%")
    return float(x)

In [69]:
id = get_model_DB_run_id_from_architecture(model_name)
id

'l679i489'

In [70]:
ENTITY = "susbrock"
PROJECT ="model_DB"

table_name = "stm32_inference_int8"

In [71]:
df = get_wandb_table_as_df(id, table_name, entity=ENTITY, project=PROJECT)
df

wandb:   1 of 1 files downloaded.  


filepath .\artifacts\run-l679i489-stm32_inference_int8-v0/stm32_inference_int8.table.json


,c_id,m_id,node_type,dur (ms),%,counters,node_name,index,"layer (type,original)",oshape,c_size,c_macc,c_type,macc,params,size,connected_to,layer_name,type
0,0,0.0,NL (0x107),0.291,0.4,62794.0,ai_node_0,1,"serving_default_input_10 (Input, )","[b:1,h:96,w:96,c:3]",,"+55,296(+100.0%)",converter_oi8[0],0,0,0,None,serving_default_input_10,"Input,"
1,None,0.1,None,NaN,NaN,NaN,None,2,"conversion_0 (Conversion, QUANTIZE)","[b:1,h:96,w:96,c:3]",,"-55,296(-100.0%)",,55296,0,0,serving_default_input_10,conversion_0,"Conversion, QUANTIZE"
2,1,1.0,Conv2dPool (0x109),7.416,11.2,1601784.0,ai_node_1,3,"conv2d_1 (Conv2D, CONV_2D)","[b:1,h:48,w:48,c:8]",,,optimized_conv2d_oi8[1],497672,224,248,conversion_0,conv2d_1,"Conv2D, CONV_2D"
3,None,1.1,None,NaN,NaN,NaN,None,4,"nl_1_nl (Nonlinearity, CONV_2D)","[b:1,h:48,w:48,c:8]",,"-18,432(-100.0%)",,18432,0,0,conv2d_1,nl_1_nl,"Nonlinearity, CONV_2D"
4,2,2.0,Pad (0x116),0.071,0.1,15426.0,ai_node_2,5,"conv2d_2 (Conv2D, DEPTHWISE_CONV_2D)","[b:1,h:48,w:48,c:8]",,,"pad_oi8/conv2d_oi8[2, 3]",165896,80,104,nl_1_nl,conv2d_2,"Conv2D, DEPTHWISE_CONV_2D"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,None,29.0,None,NaN,NaN,NaN,None,58,"reshape_29 (Reshape, RESHAPE)","[b:1,c:256]",,,,0,0,0,pool_28,reshape_29,"Reshape, RESHAPE"
71,43,30.0,NL (0x107),0.001,0.0,309.0,ai_node_43,59,"dense_30 (Dense, FULLY_CONNECTED)","[b:1,c:2]",+8(+1.6%),+6(+1.2%),"dense_oi8/converter_of32[42, 43]",512,512,512,reshape_29,dense_30,"Dense, FULLY_CONNECTED"
72,42,30.0,Dense (0x104),0.015,0.0,3264.0,ai_node_42,59,"dense_30 (Dense, FULLY_CONNECTED)","[b:1,c:2]",+8(+1.6%),+6(+1.2%),"dense_oi8/converter_of32[42, 43]",512,512,512,reshape_29,dense_30,"Dense, FULLY_CONNECTED"
73,44,31.0,NL (0x107),0.004,0.0,771.0,ai_node_44,60,"nl_31 (Nonlinearity, SOFTMAX)","[b:1,c:2]",,+4(+13.3%),"nl_of32/converter_ou8[o][44, 45]",30,0,0,dense_30,nl_31,"Nonlinearity, SOFTMAX"


In [72]:
df["macs_per_ms"]= df["macc"]/ df["dur (ms)"]
df

,c_id,m_id,node_type,dur (ms),%,counters,node_name,index,"layer (type,original)",oshape,c_size,c_macc,c_type,macc,params,size,connected_to,layer_name,type,macs_per_ms
0,0,0.0,NL (0x107),0.291,0.4,62794.0,ai_node_0,1,"serving_default_input_10 (Input, )","[b:1,h:96,w:96,c:3]",,"+55,296(+100.0%)",converter_oi8[0],0,0,0,None,serving_default_input_10,"Input,",0.000000e+00
1,None,0.1,None,NaN,NaN,NaN,None,2,"conversion_0 (Conversion, QUANTIZE)","[b:1,h:96,w:96,c:3]",,"-55,296(-100.0%)",,55296,0,0,serving_default_input_10,conversion_0,"Conversion, QUANTIZE",NaN
2,1,1.0,Conv2dPool (0x109),7.416,11.2,1601784.0,ai_node_1,3,"conv2d_1 (Conv2D, CONV_2D)","[b:1,h:48,w:48,c:8]",,,optimized_conv2d_oi8[1],497672,224,248,conversion_0,conv2d_1,"Conv2D, CONV_2D",6.710787e+04
3,None,1.1,None,NaN,NaN,NaN,None,4,"nl_1_nl (Nonlinearity, CONV_2D)","[b:1,h:48,w:48,c:8]",,"-18,432(-100.0%)",,18432,0,0,conv2d_1,nl_1_nl,"Nonlinearity, CONV_2D",NaN
4,2,2.0,Pad (0x116),0.071,0.1,15426.0,ai_node_2,5,"conv2d_2 (Conv2D, DEPTHWISE_CONV_2D)","[b:1,h:48,w:48,c:8]",,,"pad_oi8/conv2d_oi8[2, 3]",165896,80,104,nl_1_nl,conv2d_2,"Conv2D, DEPTHWISE_CONV_2D",2.336563e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,None,29.0,None,NaN,NaN,NaN,None,58,"reshape_29 (Reshape, RESHAPE)","[b:1,c:256]",,,,0,0,0,pool_28,reshape_29,"Reshape, RESHAPE",NaN
71,43,30.0,NL (0x107),0.001,0.0,309.0,ai_node_43,59,"dense_30 (Dense, FULLY_CONNECTED)","[b:1,c:2]",+8(+1.6%),+6(+1.2%),"dense_oi8/converter_of32[42, 43]",512,512,512,reshape_29,dense_30,"Dense, FULLY_CONNECTED",5.120000e+05
72,42,30.0,Dense (0x104),0.015,0.0,3264.0,ai_node_42,59,"dense_30 (Dense, FULLY_CONNECTED)","[b:1,c:2]",+8(+1.6%),+6(+1.2%),"dense_oi8/converter_of32[42, 43]",512,512,512,reshape_29,dense_30,"Dense, FULLY_CONNECTED",3.413333e+04
73,44,31.0,NL (0x107),0.004,0.0,771.0,ai_node_44,60,"nl_31 (Nonlinearity, SOFTMAX)","[b:1,c:2]",,+4(+13.3%),"nl_of32/converter_ou8[o][44, 45]",30,0,0,dense_30,nl_31,"Nonlinearity, SOFTMAX",7.500000e+03


In [73]:
summary_df = df.groupby("type").aggregate("sum")
summary_df

C:\Users\Susanne\AppData\Local\Temp\ipykernel_23812\2232791523.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,m_id,dur (ms),%,counters,index,macc,params,size,macs_per_ms
type,,,,,,,,,
"Conv2D, CONV_2D",196.0,47.382,71.3,10233870.0,406,6692312,197936,202424,2.112592e+06
"Conv2D, DEPTHWISE_CONV_2D",364.0,18.483,28.0,3993064.0,754,1599152,24800,32240,2.177871e+07
"Conversion, QUANTIZE",0.1,0.000,0.0,0.0,2,55296,0,0,0.000000e+00
"Dense, FULLY_CONNECTED",60.0,0.016,0.0,3573.0,118,1024,1024,1024,5.461333e+05
"Input,",0.0,0.291,0.4,62794.0,1,0,0,0,0.000000e+00
"Nonlinearity, CONV_2D",197.4,0.000,0.0,0.0,420,142848,0,0,0.000000e+00
"Nonlinearity, DEPTHWISE_CONV_2D",183.3,0.000,0.0,0.0,390,88704,0,0,0.000000e+00
"Nonlinearity, SOFTMAX",62.0,0.006,0.0,1309.0,120,60,0,0,2.250000e+04
"Pool, AVERAGE_POOL_2D",28.0,0.181,0.3,39124.0,57,2304,0,0,1.272928e+04


In [84]:
summary_df["macc %"] = (summary_df['macc'] / summary_df['macc'].sum()) * 100
summary_df["params %"] = (summary_df['params'] / summary_df['params'].sum()) * 100
summary_df["duration %"] = (summary_df['dur (ms)'] / summary_df['dur (ms)'].sum()) * 100
summary_df["throughput"] = (summary_df['macc'] / summary_df['dur (ms)']) 
summary_df

,m_id,dur (ms),%,counters,index,macc,params,size,macs_per_ms,macc %,params %,duration %,throughput
type,,,,,,,,,,,,,
"Conv2D, CONV_2D",196.0,47.382,71.3,10233870.0,406,6692312,197936,202424,2.112592e+06,77.983523,88.459063,71.402523,1.412417e+05
"Conv2D, DEPTHWISE_CONV_2D",364.0,18.483,28.0,3993064.0,754,1599152,24800,32240,2.177871e+07,18.634443,11.083304,27.853042,8.652015e+04
"Conversion, QUANTIZE",0.1,0.000,0.0,0.0,2,55296,0,0,0.000000e+00,0.644348,0.000000,0.000000,inf
"Dense, FULLY_CONNECTED",60.0,0.016,0.0,3573.0,118,1024,1024,1024,5.461333e+05,0.011932,0.457633,0.024111,6.400000e+04
"Input,",0.0,0.291,0.4,62794.0,1,0,0,0,0.000000e+00,0.000000,0.000000,0.438524,0.000000e+00
"Nonlinearity, CONV_2D",197.4,0.000,0.0,0.0,420,142848,0,0,0.000000e+00,1.664565,0.000000,0.000000,inf
"Nonlinearity, DEPTHWISE_CONV_2D",183.3,0.000,0.0,0.0,390,88704,0,0,0.000000e+00,1.033641,0.000000,0.000000,inf
"Nonlinearity, SOFTMAX",62.0,0.006,0.0,1309.0,120,60,0,0,2.250000e+04,0.000699,0.000000,0.009042,1.000000e+04
"Pool, AVERAGE_POOL_2D",28.0,0.181,0.3,39124.0,57,2304,0,0,1.272928e+04,0.026848,0.000000,0.272759,1.272928e+04


In [74]:
duration = df["dur (ms)"].sum()
duration

66.35900000000001

In [75]:
percentage = df["%"].sum()
percentage

100.0

In [76]:
params = df["params"].sum()
params

223760

# Visualizations

In [77]:
def create_long_df(df):
    long_df = df[["_index", 
             "dur (ms)", 
             "macc",
             #"params",
             #"size"
             ]].copy()
    long_df = pd.melt(long_df, id_vars=["_index"], var_name= "layer index", value_name="value").sort_values(by=["_index"])
    return long_df

In [78]:
long_df = create_long_df(df)
long_df

KeyError: "['_index'] not in index"

In [ ]:
def dur_macc_fig(df_long):

    #peak_memory_limit = 87
    #flash_memory_limit = 250
    fig = px.bar(df_long, 
                 x="_index", 
                 #color="size",
                 y='value',
                labels={"_index": "layer_index",
                            #"size": "size",
                            #"value": "kB"
                            },
                title="Peak memory, RAM size & flash size",
                barmode='group',
                #hover_data=['architecture', 'size'],
                height=600
                )
    # fig.add_hline(y=peak_memory_limit, line_width=2, line_dash="dash", line_color="orange", annotation_text=f"{peak_memory_limit} KB peak activation", 
    #             annotation_position="top left")
    # fig.add_hline(y=flash_memory_limit, line_width=2, line_dash="dash", line_color="blue", annotation_text=f"{flash_memory_limit} KB flash memory", 
    #             annotation_position="top right")
    fig.show()
    return fig

In [ ]:
dur_macc = dur_macc_fig(long_df)

In [ ]:
def plot_metrics_per_layer(architecture, metric):
    # #architecture_1 = "mobilenetv2_0.25_96_c3_o2_l5"
    # run_id = get_model_DB_run_id_from_architecture(architecture)
    # df = get_wandb_table_as_df(run_id, "tflite_layers_mltk")
    metrics_fig = px.bar(df, x="_index", y= metric,
                    text_auto=".2s",
                    labels={"_index" : "layer index",
                            "dur (ms)" : "duration in ms"},
                    width=1200, height=600,
                    hover_data=["node_type", "macc", "layer_name", "params", "size"],
                    title=f"{metric} per layer - tflite {architecture}")
    metrics_fig.update_traces(width=1)
    metrics_fig.update_layout(showlegend=False)

    metrics_fig.show()
    return  metrics_fig


In [ ]:
duration_fig = plot_metrics_per_layer(model_name, "dur (ms)")

In [79]:
def get_wandb_run_order_df(run_id):
    table_name = "Runorder"
    api = wandb.Api()
    artifact = api.artifact(f'{ENTITY}/{PROJECT}/run-{run_id}-{table_name}:latest')
    artifact_dir = artifact.download()
    table_path = Path(artifact_dir).joinpath("Run order.table.json")
    #print(f"filepath {table_path}")

    with open(table_path) as file:
        json_dict = json.load(file)
        #print(json_dict)
    df = pd.DataFrame(json_dict["data"], columns=json_dict["columns"])

    return df

In [80]:
run_df = get_wandb_run_order_df(id)
run_df

wandb:   1 of 1 files downloaded.  


,node type,first,avg ms,%,cdf%,mem KB,times called,Name
0,QUANTIZE,0.114,0.103,21.787,21.787,0.0,1,[tfl.quantize]:0
1,"Convolution (NHWC, QC8) IGEMM",0.044,0.047,9.826,31.613,0.0,1,"Delegate/Convolution (NHWC, QC8) IGEMM:0"
2,"Convolution (NHWC, QC8) DWConv",0.032,0.037,7.700,39.313,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:1"
3,"Convolution (NHWC, QC8) GEMM",0.024,0.027,5.698,45.011,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:2"
4,"Convolution (NHWC, QC8) DWConv",0.013,0.015,3.129,48.140,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:3"
5,"Convolution (NHWC, QC8) GEMM",0.013,0.015,3.075,51.216,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:4"
6,"Convolution (NHWC, QC8) DWConv",0.024,0.026,5.554,56.769,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:5"
7,"Convolution (NHWC, QC8) GEMM",0.019,0.021,4.515,61.284,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:6"
8,"Convolution (NHWC, QC8) DWConv",0.006,0.007,1.434,62.719,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:7"
9,"Convolution (NHWC, QC8) GEMM",0.009,0.010,2.161,64.880,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:8"


In [81]:
mltk_df = get_wandb_table_as_df(id, "tflite_INt8_layers_mltk")
mltk_df

wandb:   1 of 1 files downloaded.  


filepath .\artifacts\run-l679i489-tflite_INt8_layers_mltk-v0/tflite_INt8_layers_mltk.table.json


,index,opcode,name,ops,macs,cpu_cycles,energy,input_shape,output_shape,options
0,0,quantize,0: quantize,110592,0,9.957829e+05,1.988387e-04,1x96x96x3,1x96x96x3,Type=none
1,1,conv_2d,1: conv_2d,1050624,497664,4.600492e+06,8.359018e-04,"1x96x96x3,8x3x3x3,8",1x48x48x8,Padding:same stride:2x2 activation:relu
2,2,depthwise_conv_2d,2: depthwise_conv_2d,387072,165888,2.522040e+06,4.679793e-04,"1x48x48x8,1x3x3x8,8",1x48x48x8,Multiplier:1 padding:same stride:1x1 activation:relu
3,3,conv_2d,3: conv_2d,700416,294912,3.560665e+06,6.482228e-04,"1x48x48x8,16x1x1x8,16",1x48x48x16,Padding:same stride:1x1 activation:relu
4,4,depthwise_conv_2d,4: depthwise_conv_2d,193536,82944,1.036370e+06,2.085251e-04,"1x48x48x16,1x3x3x16,16",1x24x24x16,Multiplier:1 padding:same stride:2x2 activation:relu
5,5,conv_2d,5: conv_2d,645120,294912,1.691265e+06,2.639363e-04,"1x24x24x16,32x1x1x16,32",1x24x24x32,Padding:same stride:1x1 activation:relu
6,6,depthwise_conv_2d,6: depthwise_conv_2d,387072,165888,2.052064e+06,3.942758e-04,"1x24x24x32,1x3x3x32,32",1x24x24x32,Multiplier:1 padding:same stride:1x1 activation:relu
7,7,conv_2d,7: conv_2d,1234944,589824,2.210628e+06,3.351090e-04,"1x24x24x32,32x1x1x32,32",1x24x24x32,Padding:same stride:1x1 activation:relu
8,8,depthwise_conv_2d,8: depthwise_conv_2d,96768,41472,5.061695e+05,9.911860e-05,"1x24x24x32,1x3x3x32,32",1x12x12x32,Multiplier:1 padding:same stride:2x2 activation:relu
9,9,conv_2d,9: conv_2d,617472,294912,1.101254e+06,1.183100e-04,"1x12x12x32,64x1x1x32,64",1x12x12x64,Padding:same stride:1x1 activation:relu
